<a href="https://colab.research.google.com/github/ShivamCholin/CS6910_Assignment_1/blob/main/NAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random          
import numpy as np      
from time import time    

In [2]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))
def dsigmoid(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [3]:
def relu(z):
    return np.maximum(z, 0)
def drelu(z):
    return np.heaviside(z, 1)

In [4]:
def tanh(z):
    return (np.exp(z)-np.exp(-z))/(np.exp(z)+np.exp(-z))
def dtanh(z):
    return 1-tanh(z)**2

In [5]:
def naglearn(nn, tr_data, epochs, batch_size, learning_rate):
    n = len(tr_data)
    for j in range(epochs):
        time_start=time()
        random.shuffle(tr_data)
        batches = [tr_data[k: k + batch_size] for k in range(0,n,batch_size)]
        for batch in batches:
            mbgd(nn, batch, learning_rate)
        time_end=time()
        print('Epoch {0}:time taken {1} seconds, accuracy {2}%'.format(f'{j + 1:2}',1.0*time_end-time_start, 100.0 * evaluate2(nn, tr_data) / len(tr_data)))


In [6]:
def nag(nn, batch, eta):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    for x, y in batch:
        dnb, dnw = backward(nn, x, y) 

        nb = [nb + dnb for nb, dnb in zip(nb, dnb)]
        nw = [nw + dnw for nw, dnw in zip(nw, dnw)]

    nn.vw = [vweight*nn.gamma + eta * nw for vweight,nw in zip(nn.vw,nw)]
    nn.vb = [vbiases*nn.gamma + eta * nb for vbiases,nb in zip(nn.vb,nb)]
    #nn.weights = [w - (eta ) * nw for w, nw in zip(nn.weights, nw)]
    #nn.biases  = [b - (eta ) * nb for b, nb in zip(nn.biases, nb)]
    nn.weights = [w - v for w, v in zip(nn.weights, nn.vw)]
    nn.biases  = [b - v for b, v in zip(nn.biases, nn.vb)]

In [7]:
def dcost(act, y):
    return (act - y)

In [8]:
class Network:
    num_layers=0
    biases=[]
    weights=[]
    def __init__(self,nl,x,y,act1,act2,vb,vw):
      self.num_layers=nl
      self.biases=x
      self.weights=y
      self.act=act1
      self.dact=act2
      self.vw=vw
      self.vb=vb
      self.gamma=0.9

def init_network(layers,actfunc):
    if actfunc=="tanh":
      act1=tanh
      act2=dtanh
    elif actfunc=="sigmoid":
      act1=sigmoid
      act2=dsigmoid
    else:
      act1=relu
      act2=drelu
    return Network(len(layers),[np.random.randn(y, 1) for y in layers[1:]],[np.random.randn(y, x) for x, y in zip(layers[:-1], layers[1:])],act1,act2,[np.random.randn(y, 1)*0 for y in layers[1:]],[np.random.randn(y, x)*0 for x, y in zip(layers[:-1], layers[1:])])

In [9]:
def forward(nn, a):
    for b, w in zip(nn.biases, nn.weights):
        a = nn.act(np.dot(w, a) + b)
    return a

In [10]:
def evaluate(nn, te_data):
    test_results = [(np.argmax(forward(nn, x)), y) for (x, y) in te_data]
    return sum(int(x == y) for (x, y) in test_results)

In [11]:
def evaluate2(nn, te_data):
    test_results = [(np.argmax(forward(nn, x)), y) for (x, y) in te_data]
    return sum(int(x == np.argmax(y)) for (x, y) in test_results)

In [14]:
def backward(nn, x, y):
    nb = [np.zeros(b.shape) for b in nn.biases]
    nw = [np.zeros(w.shape) for w in nn.weights]
    activation = x 
    acts = [x]
    zs = []  
    new_weights = [w-v*nn.gamma for w,v in zip(nn.weights,nn.vw)]
    new_biases = [b-v*nn.gamma for b,v in zip(nn.biases,nn.vb)]
    for b, w in zip(new_biases, new_weights):
        z = np.dot(w, activation) + b 
        zs.append(z)        
        activation = nn.act(z)   
        acts.append(activation)
    delta = dcost(acts[-1], y) * nn.dact(zs[-1])
    nb[-1] = delta
    nw[-1] = np.dot(delta, acts[-2].transpose())
    for i in range(2, nn.num_layers):
        z = zs[-i]
        sp = nn.dact(z)
        delta = np.dot(new_weights[-i + 1].transpose(), delta) * sp
        nb[-i] = delta
        nw[-i] = np.dot(delta, acts[-i - 1].transpose())
    return (nb, nw)

In [15]:
import keras
def change_y(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e
fashion_mnist = keras.datasets.fashion_mnist
tr_data, te_data = fashion_mnist.load_data()
training_x = [np.reshape(x, (784, 1))/255 for x in tr_data[0]]
training_y = [change_y(y) for y in tr_data[1]]
tr_data = zip(training_x, training_y)
test_inputs = [np.reshape(x, (784, 1))/255 for x in te_data[0]]
te_data = zip(test_inputs, te_data[1])
tr_data=list(tr_data)
te_data=list(te_data)

In [17]:
nn = init_network([784, 30, 10],"tanh")
epochs = 20
batch_size =10
learning_rate = 0.001
print('start')
mbgdlearn(nn, tr_data, epochs, batch_size, learning_rate)
print('accuracy {0}%'.format(100.0 * evaluate(nn, te_data) / len(te_data)))

start
Epoch  1:time taken 16.02422285079956 seconds, accuracy 53.108333333333334%
Epoch  2:time taken 15.998562812805176 seconds, accuracy 58.30833333333333%
Epoch  3:time taken 16.01942729949951 seconds, accuracy 60.52333333333333%
Epoch  4:time taken 19.199312686920166 seconds, accuracy 59.181666666666665%
Epoch  5:time taken 16.05104684829712 seconds, accuracy 58.428333333333335%
Epoch  6:time taken 16.150882244110107 seconds, accuracy 62.175%
Epoch  7:time taken 16.17069363594055 seconds, accuracy 62.57666666666667%
Epoch  8:time taken 16.09942936897278 seconds, accuracy 62.196666666666665%
Epoch  9:time taken 16.071519374847412 seconds, accuracy 62.968333333333334%
Epoch 10:time taken 16.106752395629883 seconds, accuracy 63.255%
Epoch 11:time taken 16.020867824554443 seconds, accuracy 62.858333333333334%
Epoch 12:time taken 16.204726696014404 seconds, accuracy 63.63166666666667%
Epoch 13:time taken 15.985183000564575 seconds, accuracy 60.635%
Epoch 14:time taken 16.212037801742554